<a href="https://www.kaggle.com/code/zainabs1/marketing-campaign-analysis-insights-with-sql?scriptVersionId=291461587" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install duckdb

In [2]:
#import necessary libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns

## Dataset Loading

In [3]:
#load the dataset
df = pd.read_csv("../input/marketing-campaign/marketing_campaign.csv")

In [4]:
#inspecting the dataset
df.head()

,Unnamed: 0,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,...,7,0,0,0,0,0,0,3,11,1
1,1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,...,5,0,0,0,0,0,0,3,11,0
2,2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,...,4,0,0,0,0,0,0,3,11,0
3,3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,...,6,0,0,0,0,0,0,3,11,0
4,4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,...,5,0,0,0,0,0,0,3,11,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           2240 non-null   int64  
 1   ID                   2240 non-null   int64  
 2   Year_Birth           2240 non-null   int64  
 3   Education            2240 non-null   object 
 4   Marital_Status       2240 non-null   object 
 5   Income               2216 non-null   float64
 6   Kidhome              2240 non-null   int64  
 7   Teenhome             2240 non-null   int64  
 8   Dt_Customer          2240 non-null   object 
 9   Recency              2240 non-null   int64  
 10  MntWines             2240 non-null   int64  
 11  MntFruits            2240 non-null   int64  
 12  MntMeatProducts      2240 non-null   int64  
 13  MntFishProducts      2240 non-null   int64  
 14  MntSweetProducts     2240 non-null   int64  
 15  MntGoldProds         2240 non-null   i

In [6]:
df.describe()

,Unnamed: 0,ID,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
count,2240.000000,2240.000000,2240.000000,2216.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,...,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.0,2240.0,2240.000000
mean,1119.500000,5592.159821,1968.805804,52247.251354,0.444196,0.506250,49.109375,303.935714,26.302232,166.950000,...,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.009375,3.0,11.0,0.149107
std,646.776623,3246.662198,11.984069,25173.076661,0.538398,0.544538,28.962453,336.597393,39.773434,225.715373,...,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.096391,0.0,0.0,0.356274
min,0.000000,0.000000,1893.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
25%,559.750000,2828.250000,1959.000000,35303.000000,0.000000,0.000000,24.000000,23.750000,1.000000,16.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
50%,1119.500000,5458.500000,1970.000000,51381.500000,0.000000,0.000000,49.000000,173.500000,8.000000,67.000000,...,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
75%,1679.250000,8427.750000,1977.000000,68522.000000,1.000000,1.000000,74.000000,504.250000,33.000000,232.000000,...,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
max,2239.000000,11191.000000,1996.000000,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,...,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.0,11.0,1.000000


In [7]:
#checking for missing values
df.isnull().sum()

Unnamed: 0              0
ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64

In [8]:
#checking for duplicates
df.duplicated().sum()

0

## Data Cleaning

### 1. Fixing Naming Convention

In [9]:
print(df.columns.tolist())

['Unnamed: 0', 'ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response']


In [10]:
new_column_names = ['index', 'customer_id', 'birth_year', 'education_level', 'marital_status', 'income', 'kids_count', 'teens_count', 'enrollment_date', 'recency', 'spent_wines', 'spent_fruits', 'spent_meat', 'spent_fish', 'spent_sweet', 'spent_gold', 'purchase_discount', 'purchase_web', 'purchase_catalog', 'purchase_store', 'web_visit_per_month', 'accepted_campaign3', 'accepted_campaign4', 'accepted_campaign5', 'accepted_campaign1', 'accepted_campaign2', 'complained', 'contact_cost', 'campaign_revenue', 'accepted_campaign6']
df.columns = new_column_names

***Note***: We assumed `Response` to be customer responses to the last campaign, i.e., the **6th campaign**, and therefore changed its name to `accepted_campaign6` instead of `Response`.

### 2. Fixing Data Types

In [11]:
#checking column data types
df.dtypes

index                    int64
customer_id              int64
birth_year               int64
education_level         object
marital_status          object
income                 float64
kids_count               int64
teens_count              int64
enrollment_date         object
recency                  int64
spent_wines              int64
spent_fruits             int64
spent_meat               int64
spent_fish               int64
spent_sweet              int64
spent_gold               int64
purchase_discount        int64
purchase_web             int64
purchase_catalog         int64
purchase_store           int64
web_visit_per_month      int64
accepted_campaign3       int64
accepted_campaign4       int64
accepted_campaign5       int64
accepted_campaign1       int64
accepted_campaign2       int64
complained               int64
contact_cost             int64
campaign_revenue         int64
accepted_campaign6       int64
dtype: object

In [12]:
#columns that should be `Date` type
df['enrollment_date'] = pd.to_datetime(df['enrollment_date'], format='%Y-%m-%d')

### 3. Handling Missing Values

In [13]:
#fill missing values in the Income column
df['income'] = df['income'].fillna(df['income'].median())

In [14]:
#confirming
df.isnull().sum()

index                  0
customer_id            0
birth_year             0
education_level        0
marital_status         0
income                 0
kids_count             0
teens_count            0
enrollment_date        0
recency                0
spent_wines            0
spent_fruits           0
spent_meat             0
spent_fish             0
spent_sweet            0
spent_gold             0
purchase_discount      0
purchase_web           0
purchase_catalog       0
purchase_store         0
web_visit_per_month    0
accepted_campaign3     0
accepted_campaign4     0
accepted_campaign5     0
accepted_campaign1     0
accepted_campaign2     0
complained             0
contact_cost           0
campaign_revenue       0
accepted_campaign6     0
dtype: int64

## Feature Enigneering
Here are the changes we're going to make:
1. **Create `customer_age` from `birth_year`**. Age is more interpretable and supports segmentation.
3. Instead of `kids_count` and `teens_count`, we are summing them into `children`.
4. **We'll use `children` to derive `is_parent`**.
5. `children` and `marital_status` will be used to generate `family_size`.
6. **Convert `enrollment_date` to `customer_for`**. Measuring how long a customer has been with the company is useful for loyalty analysis.
7. **Sum all spents columns into `spent`**
8. Drop unnecessary columns.

In [15]:
#derive customer_age
from datetime import datetime

df['customer_age'] = datetime.now().year - df['birth_year']

In [16]:
#sum kids_count and teens_count into children
df['children'] = df['kids_count'] + df['teens_count']

In [17]:
#derive is_parent
df['is_parent'] = np.where(df['children']>0, 1, 0)

In [18]:
#derive family_size
df['with_partner'] = np.where((df['marital_status'] == 'Married') | (df['marital_status'] == 'Together'), 1, 0)
df['family_size'] = df['children'] + df['with_partner'] + 1

Now, to create the `customer_for` feature, we will need to give some special consideration. Since the dataset is a bit old, we will be using **the last date in the dataset** instead of current date.

In [19]:
#fetch last date in the dataset
last_date = max(df['enrollment_date'])
last_date

Timestamp('2014-06-29 00:00:00')

In [20]:
df['customer_for'] = last_date - df['enrollment_date']

In [21]:
#sum all spent categories into total spent
df['total_spent'] = df['spent_wines'] + df['spent_fruits'] + df['spent_meat'] + df['spent_fish'] + df['spent_sweet'] + df['spent_gold']

In [22]:
#drop unnecessary columns
df.drop(['index', 'birth_year', 'kids_count', 'teens_count', 'enrollment_date', 'contact_cost', 'campaign_revenue'], axis=1,inplace=True)

At the end, this is the list of features we have:

In [23]:
display(df.columns.tolist())

['customer_id',
 'education_level',
 'marital_status',
 'income',
 'recency',
 'spent_wines',
 'spent_fruits',
 'spent_meat',
 'spent_fish',
 'spent_sweet',
 'spent_gold',
 'purchase_discount',
 'purchase_web',
 'purchase_catalog',
 'purchase_store',
 'web_visit_per_month',
 'accepted_campaign3',
 'accepted_campaign4',
 'accepted_campaign5',
 'accepted_campaign1',
 'accepted_campaign2',
 'complained',
 'accepted_campaign6',
 'customer_age',
 'children',
 'is_parent',
 'with_partner',
 'family_size',
 'customer_for',
 'total_spent']

## Data Model
We will first create the SQL engine, to which we will upload the data for analysis.

In [24]:
#create the SQL engine
engine = create_engine("sqlite:///marketing.db")

### 1. fact_campaign_responses
For better analysis,  we are going to **Unpivot** the campaign acceptence columns.

In [25]:
#identify columns to unpivot
campaign_columns = [
    'accepted_campaign1',
    'accepted_campaign2',
    'accepted_campaign3',
    'accepted_campaign4',
    'accepted_campaign5',
    'accepted_campaign6'
]

In [26]:
campaign_responses = df.melt(
    id_vars=['customer_id'],       #keep
    value_vars=campaign_columns,      #columns to unpivot
    var_name='campaign_id',
    value_name='accepted'
)

In [27]:
#extract numeric campaign_id
campaign_responses['campaign_id'] = campaign_responses['campaign_id'].str.extract('(\d+)').astype(int)

In [28]:
#upload to SQL
campaign_responses.to_sql('campaign_responses', engine, index=False, if_exists='replace')

13440

### 2. dim_customer

In [29]:
#customer dimension table
customer = df[['customer_id', 'education_level', 'marital_status', 'customer_age', 'with_partner', 'is_parent', 'children', 'family_size', 'customer_for']].drop_duplicates()
#upload to SQL
customer.to_sql('customer', engine, index=False, if_exists='replace')

/tmp/ipykernel_13/1579010461.py:4: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  customer.to_sql('customer', engine, index=False, if_exists='replace')


2240

### 2. fact_customer_activities

In [30]:
#customer dimension table
customer_activities = df[['customer_id', 'income', 'recency', 'spent_wines', 'spent_fruits', 'spent_meat', 'spent_fish', 'spent_sweet', 'spent_gold', 'total_spent', 'purchase_discount', 'purchase_web', 'purchase_catalog', 'purchase_store', 'web_visit_per_month', 'complained']].drop_duplicates()
#upload to SQL
customer_activities.to_sql('customer_activities', engine, index=False, if_exists='replace')

2240

## Data Analysis with SQL
Let's check the row counts first.

In [31]:
#row counts
total_customer = pd.read_sql("SELECT COUNT(*) AS total_customer FROM customer", engine)
total_activities = pd.read_sql("SELECT COUNT(*) AS total_activities FROM customer_activities", engine)
total_responses = pd.read_sql("SELECT COUNT(*) AS total_responses FROM campaign_responses", engine)

display(total_customer)
display(total_activities)
display(total_responses)

,total_customer
0,2240


,total_activities
0,2240


,total_responses
0,13440


### 1. Customer Profiling

In [32]:
#age distribution
query_a1 = """
SELECT 
    CASE 
        WHEN customer_age < 25 THEN 'Under 25'
        WHEN customer_age BETWEEN 25 AND 34 THEN '25–34'
        WHEN customer_age BETWEEN 35 AND 44 THEN '35–44'
        WHEN customer_age BETWEEN 45 AND 54 THEN '45–54'
        WHEN customer_age BETWEEN 55 AND 64 THEN '55–64'
        ELSE '65+'
    END AS age_bucket,
    COUNT(*) AS customer_count
FROM customer
GROUP BY age_bucket
ORDER BY age_bucket ASC
"""
result_a1 = pd.read_sql(query_a1, engine)
display(result_a1)

,age_bucket,customer_count
0,25–34,28
1,35–44,318
2,45–54,654
3,55–64,585
4,65+,655


In [33]:
#education level distribution
query_a2 = """
SELECT 
    education_level, COUNT(*) AS customer_count
FROM customer
GROUP BY education_level
ORDER BY customer_count DESC
"""
result_a2 = pd.read_sql(query_a2, engine)
display(result_a2)

,education_level,customer_count
0,Graduation,1127
1,PhD,486
2,Master,370
3,2n Cycle,203
4,Basic,54


In [34]:
#marital status distribution
query_a3 = """
SELECT 
    marital_status, COUNT(*) AS customer_count
FROM customer
GROUP BY marital_status
ORDER BY customer_count DESC
"""
result_a3 = pd.read_sql(query_a3, engine)
display(result_a3)

,marital_status,customer_count
0,Married,864
1,Together,580
2,Single,480
3,Divorced,232
4,Widow,77
5,Alone,3
6,YOLO,2
7,Absurd,2


***Insights***:
1. The majority of customers fall within the `35-64` age range, with the largest bucket being `45-54` years old (702 customers). This indicates that the customer base is primarily **middle-aged**.
2. The most common education levels are `Graduation (Bachelor’s)` (1,127 customers), `PhD` (486 customers), and `Master’s` (370 customers). This means the customer base is **highly educated**. This typically correlates with *higher income levels*.
3. More than half of customers fall into two marital status, `Married` (864 customers) and `Together` (living with partner) (580 customers). This shows that the customer base is predominantly **in committed relationships or family**, which may directly influences their spending patterns and consumption.

Combining all three, our **primary persona** (dominant customer segment) is:

**A middle-aged (45-54), highly educated customer, who's married or living with a partner**.

This is the segment representing the largest share of our customer base and is likely to drive the majority of spending activities.

*This dominant segment will be the primary focus of the subsequent behavioral and spending analysis.*

### 2. Spending Analysis

In [35]:
#spending distribution
query_b1 = """
WITH buckets AS (
    SELECT
        customer_id,
        total_spent,
        NTILE(5) OVER (ORDER BY total_spent) AS spending_bucket
    FROM customer_activities
)
SELECT 
    spending_bucket,
    COUNT(*) AS customer_count,
    MIN(total_spent) AS min_spent,
    MAX(total_spent) AS max_spent,
    ROUND(AVG(total_spent), 2) AS avg_spent
FROM buckets
GROUP BY spending_bucket
ORDER BY spending_bucket;
"""

result_b1 = pd.read_sql(query_b1, engine)
display(result_b1)

,spending_bucket,customer_count,min_spent,max_spent,avg_spent
0,1,448,5,55,33.60
1,2,448,55,194,101.91
2,3,448,195,635,392.59
3,4,448,636,1174,915.96
4,5,448,1174,2525,1584.94


***Insights***:
* Average spending increases exponentially, not linearly; whihc means that we have **strong spending inequality**.
* **Top 20% bucket is very valuable**. A customer in `bucket 5` spends ~ 1584 on average, which is **48x** more than a customer in `bucket 1` (bottom 20%). This indicates a **Pareto behavior (80/20 rule)** *— revenue is heavily driven by high spenders.*
* We can cluster spending into **3 groups**: **Low spenders** (`buckets 1-2`), **Mid spenders** (`bucket 3`), and **High spenders** (`buckets 4-5`).

Therefore — **high spenders** should be prioritized; since they account for most revenue. **Mid spenders** can be converted to high spenders; since they are close to buckets 4. Lastly, **low spenders** need to be activated through campaigns.

In [36]:
#average spending by age buckets
query_b2 = """
WITH age_buckets AS (
    SELECT
        c.customer_id,
        a.total_spent,
        CASE 
            WHEN c.customer_age < 25 THEN 'Under 25'
            WHEN c.customer_age BETWEEN 25 AND 34 THEN '25–34'
            WHEN c.customer_age BETWEEN 35 AND 44 THEN '35–44'
            WHEN c.customer_age BETWEEN 45 AND 54 THEN '45–54'
            WHEN c.customer_age BETWEEN 55 AND 64 THEN '55–64'
            ELSE '65+'
        END AS age_bucket
    FROM customer c
        JOIN customer_activities a
        ON c.customer_id = a.customer_id
)
SELECT
    age_bucket,
    COUNT(*) AS customer_count,
    ROUND(AVG(total_spent), 2) AS avg_spent,
    ROUND(SUM(total_spent), 2) AS total_segment_revenue
FROM age_buckets
GROUP BY age_bucket
ORDER BY age_bucket ASC;
"""

result_b2 = pd.read_sql(query_b2, engine)
display(result_b2)

,age_bucket,customer_count,avg_spent,total_segment_revenue
0,25–34,28,828.00,23184.0
1,35–44,318,570.92,181554.0
2,45–54,654,500.89,327584.0
3,55–64,585,608.16,355774.0
4,65+,655,715.87,468892.0


***Insights***:
* Younger customer spends the most *per person*. Even though they are a very small group (43 person), they have the **highest average spending**, but because the group is small, they contribute low total revenue.
* `45-54` is the largest groups *but not* the highest spenders. Although they are a large segment, they are a moderate-value one.
* The `65+` is the most valuable segment. They combine all **high count**, **high average spend**, and ***highest* total revenue**.

Even though younger customers spend more individually — the 45-64 and especially the 65+ groups represent the core revenue engine. 

In [37]:
#spending by education level
query_b3 = """
SELECT
    c.education_level,
    COUNT(*) AS customer_count,
    ROUND(AVG(a.total_spent), 2) AS avg_spent,
    ROUND(SUM(a.total_spent), 2) AS total_segment_revenue
FROM customer c
    JOIN customer_activities a
    ON c.customer_id = a.customer_id
GROUP BY c.education_level
ORDER BY avg_spent DESC;
"""

result_b3 = pd.read_sql(query_b3, engine)
display(result_b3)

,education_level,customer_count,avg_spent,total_segment_revenue
0,PhD,486,672.41,326791.0
1,Graduation,1127,619.90,698626.0
2,Master,370,611.78,226359.0
3,2n Cycle,203,496.53,100795.0
4,Basic,54,81.80,4417.0


***Inisghts***:
* Highly educated customers spend more. `Graducation`, `Master's`, and `PhD` degrees have *higher average spend*, while the *lowest spending* is observed in customers with `Basic` education.
* Largest revenue comes from the `Graduation` group (total revenue 698,626), which has the largest number of customers (1,127).
* Low-education customers contribute very little.


Education level correlates positively with individual spending power. Therefore — we should target customers with **high educations**.

In [38]:
#spending by marital status
query_b4 = """
SELECT
    c.marital_status,
    COUNT(*) AS customer_count,
    ROUND(AVG(a.total_spent), 2) AS avg_spent,
    ROUND(SUM(a.total_spent), 2) AS total_segment_revenue
FROM customer c
    JOIN customer_activities a
    ON c.customer_id = a.customer_id
GROUP BY c.marital_status
ORDER BY avg_spent DESC;
"""

result_b4 = pd.read_sql(query_b4, engine)
result_b4

,marital_status,customer_count,avg_spent,total_segment_revenue
0,Absurd,2,1192.50,2385.0
1,Widow,77,738.82,56889.0
2,Divorced,232,610.63,141666.0
3,Together,580,608.39,352865.0
4,Single,480,606.48,291112.0
5,Married,864,590.80,510453.0
6,YOLO,2,424.00,848.0
7,Alone,3,256.67,770.0


***Insights***:
* The highest averaeg spend per person comes from the `Widow` and `Divorced` segments (neglecting **Absurd** since they are only 3 customers).
* `Married`, `Together`, and `Single` segments contribute the highest total revenue. So, even though they spend less individually, large segments drive the total revenue.
* `Absurd`, `YOLO`, and `Alone` are negligible, since they are too small to impact revenue.

However — although the average spending appear to be higher in the `Widow` and `Divorced` segments, this seems to be misleading; because these groups are smaller. So, without considering household composition, actual spending behavior is not secured. That's why we will need to analyze the **presence of children**, since they are a critical contributing factor.

In [39]:
#spending by marital status and children (family size)
query_b5 = """
WITH customer_children AS (
    SELECT
        c.customer_id,
        c.marital_status,
        c.children,
        a.total_spent,
        CASE 
            WHEN children = 0 THEN '0 Children'
            WHEN children = 1 THEN '1 Child'
            WHEN children = 2 THEN '2 Children'
            ELSE '3+ Children'
        END AS children_bucket
    FROM customer c
        JOIN customer_activities a
        ON c.customer_id = a.customer_id
    WHERE c.marital_status IN ('Widow', 'Divorced', 'Married', 'Together')
)
SELECT
    marital_status,
    children_bucket,
    COUNT(*) AS customer_count,
    ROUND(AVG(total_spent), 2) AS avg_spent,
    ROUND(SUM(total_spent), 2) AS total_segment_revenue
FROM customer_children
GROUP BY marital_status, children_bucket
ORDER BY marital_status, children_bucket;
"""

result_b5 = pd.read_sql(query_b5, engine)
result_b5

,marital_status,children_bucket,customer_count,avg_spent,total_segment_revenue
0,Divorced,0 Children,58,1009.59,58556.0
1,Divorced,1 Child,120,580.37,69644.0
2,Divorced,2 Children,49,265.31,13000.0
3,Divorced,3+ Children,5,93.20,466.0
4,Married,0 Children,230,1049.00,241270.0
5,Married,1 Child,449,492.93,221325.0
6,Married,2 Children,168,253.02,42508.0
7,Married,3+ Children,17,314.71,5350.0
8,Together,0 Children,154,1192.24,183605.0
9,Together,1 Child,299,458.27,137023.0


***Insights***:
* In every segment, highest averaeg spenders are **childless customers** — this explains why `Widow` and `Divorced` segments appeared to spend more on average previously.
* **Adding children *reduces* average spending in *all* segments**.
* Lastly, most childless customers are smaller in count; small number + highe spending → artificially high average.

To conclude — the initial "**`Widow` / `Divorced` spend more on average**" conclusion was for sure *misleading*. While childless `Widow` and `Divorced` customers showed high average spending, they are very small. When taking children into account, the higher average spending in these segments is explained by dominance of small childless households. So, the **largest revenue contributors** remain `Married` and `Together` households **with children**.

***Note***: Deeper behavior analysis will be condcuted..